ich_locator


In [1]:

import tensorflow as tf 
import pandas as pd
from utils import convert, extract, registration

In [ ]:
data_path = 
table_path = 

In [ ]:

data_path = '/home/'

os.chdir(data_path)
files = sorted(os.listdir('.'))

os.chdir(data_path)
files = sorted(next(os.walk(data_path))[2])

images = [f for f in files if '.nii.gz' in f]
  
names = sorted(list(set([f[:7] for f in images])))

image_set = []

for name in names:
    subject_images = [f for f in images if f[:7] == name]
    image = subject_images[0]
    image_set.append(image)

            
images = image_set

df = pd.DataFrame(columns = ['subject', 'deep', 'lobar', 'infra']) 

df['subject'] = names

df.head(len(names))

In [ ]:
# load the excel files, search for the cell for each name with lobar etc. and put in the new table

In [2]:

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  #if isinstance(value, type(tf.constant(0))):
  #      value = value.numpy()
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# Create the serialized example of features
def serialize_example(feature0, feature1, feature2, feature3):
    
    feature = {'deep': _int64_feature(feature0),
               'lobar': _int64_feature(feature1),
               'infra': _int64_feature(feature2),
               'image': _bytes_feature(tf.compat.as_bytes(feature3))}
    
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()


In [ ]:

template = nib.load('/home/datasets/tfrecords_conversion_old/templates/scct_unsmooth_SS_0.01_128x128x128.nii.gz')
template = convert.nii2ants(template)

def tfrecords_conversion(filename, data_path, images):
    
    # Open the writer 
    writer = tf.io.TFRecordWriter(filename)
    
    # Iterate over files
    for i in range(len(images)): 

        image = nib.load(os.path.join(data_path,images[i]))
        image = convert.nii2ants(image)
        image, transforms = registration.rigid(template, image)
        image = image.numpy() #look at ants docs
        
        print(np.amin(image), np.amax(image))
        
        #find the values for each subject here to serialize
        lobar_val = df.loc(subject==).value()
        deep_val = df.loc
        infra_val = df.loc
        
        example = serialize_example(deep_val, lobar_val, infra_val, image.tostring())
    
        # Serialize to string and write to file
        writer.write(example)
        sys.stdout.flush()  # to read any intermediate error messages

    writer.close() 


In [ ]:
tfrecords_conversion(filename, data_path, images)

In [ ]:
        shape = image.shape
        new_shape = (128, 128, 128)
        image = image.astype(np.float32)
        image =  scipy.ndimage.zoom(image, (new_shape[0]/shape[0], new_shape[1]/shape[1], new_shape[2]/shape[2]), order=0)
        print(image.shape)
        
        if image.shape != new_shape:
            print("FAIL", image.shape, images[i])
            continue